In [4]:
# !pip install transformers torch gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 3.1 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.3 MB/s eta 0:00:0000:0100:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 4.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 2.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.8 MB/s eta 0:00:0000:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━

In [5]:
import os
import json
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch.utils.data import Dataset, DataLoader
import gradio as gr
import zipfile
import pickle
import MeCab

/home/user/miniforge3/envs/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
import tarfile

# .tar 파일 경로 지정
tar_file_path = '/home/user/Downloads/download.tar'

# .tar 파일을 추출할 디렉토리 지정
extract_to = './extracted_files'

# 추출할 디렉토리가 없으면 생성
os.makedirs(extract_to, exist_ok=True)

# .tar 파일 열기 및 추출
with tarfile.open(tar_file_path, 'r') as tar:
    tar.extractall(path=extract_to)
    print(f"파일이 {extract_to} 경로에 성공적으로 추출되었습니다.")


파일이 ./extracted_files 경로에 성공적으로 추출되었습니다.


In [ ]:
# import os
# import zipfile

# # Training과 Validation 경로 설정
# train_label_data_path = './extracted_files/023.국회_회의록_기반_지식검색_데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터'  # Training 하위의 라벨링 데이터 경로
# val_label_data_path = './extracted_files/023.국회_회의록_기반_지식검색_데이터/3.개방데이터/1.데이터/Validation/02.라벨링데이터'  # Validation 하위의 라벨링 데이터 경로

# # 압축을 풀 폴더 설정
# train_extracted_path = './extracted_files/Training'  # Training 데이터 압축 해제 경로
# val_extracted_path = './extracted_files/Validation'  # Validation 데이터 압축 해제 경로

# # ZIP 파일 압축 해제 함수
# def extract_zip_files(zip_folder, extract_to):
#     zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]  # 폴더 내 zip 파일만 필터링
#     for zip_file in zip_files:
#         zip_file_path = os.path.join(zip_folder, zip_file)
#         with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#             zip_ref.extractall(extract_to)
#             print(f"{zip_file} 압축이 성공적으로 해제되었습니다.")

# # Training 라벨링 데이터 압축 해제
# extract_zip_files(train_label_data_path, train_extracted_path)

# # Validation 라벨링 데이터 압축 해제
# extract_zip_files(val_label_data_path, val_extracted_path)


In [8]:
# Training과 Validation 경로 설정
train_part_folder = './extracted_files/023.국회_회의록_기반_지식검색_데이터/3.개방데이터/1.데이터/Training/02.라벨링데이터'
val_part_folder = './extracted_files/023.국회_회의록_기반_지식검색_데이터/3.개방데이터/1.데이터/Validation/02.라벨링데이터'

# 병합된 ZIP 파일이 저장될 경로 설정
train_zip_output_folder = './extracted_files/Training'
val_zip_output_folder = './extracted_files/Validation'

# 병합된 ZIP 파일 저장 경로 설정
train_output_zip_files = {
    "TL_국정감사.zip": os.path.join(train_zip_output_folder, "TL_국정감사.zip"),
    "TL_본회의.zip": os.path.join(train_zip_output_folder, "TL_본회의.zip"),
    "TL_소위원회.zip": os.path.join(train_zip_output_folder, "TL_소위원회.zip"),
    "TL_예산결산특별위원회.zip": os.path.join(train_zip_output_folder, "TL_예산결산특별위원회.zip"),
    "TL_특별위원회.zip": os.path.join(train_zip_output_folder, "TL_특별위원회.zip"),
}

val_output_zip_files = {
    "VL_국정감사.zip": os.path.join(val_zip_output_folder, "VL_국정감사.zip"),
    "VL_본회의.zip": os.path.join(val_zip_output_folder, "VL_본회의.zip"),
    "VL_소위원회.zip": os.path.join(val_zip_output_folder, "VL_소위원회.zip"),
    "VL_예산결산특별위원회.zip": os.path.join(val_zip_output_folder, "VL_예산결산특별위원회.zip"),
    "VL_특별위원회.zip": os.path.join(val_zip_output_folder, "VL_특별위원회.zip"),
}

# 1. 분할 파일을 병합하는 함수
def merge_part_files(part_folder, output_file_prefix):
    part_files = sorted([f for f in os.listdir(part_folder) if f.endswith('.part0')], key=lambda x: x)
    for part_file in part_files:
        zip_file_key = part_file.replace('.zip.part0', '.zip')  # 중복 확장자 방지
        if zip_file_key in output_file_prefix:
            zip_file = output_file_prefix[zip_file_key]

            # 폴더가 존재하지 않으면 생성
            os.makedirs(os.path.dirname(zip_file), exist_ok=True)
            
            part_file_path = os.path.join(part_folder, part_file)
            with open(zip_file, 'wb') as merged_file:
                with open(part_file_path, 'rb') as pf:
                    merged_file.write(pf.read())
                print(f"{part_file} 병합 완료. ZIP 파일 저장 경로: {zip_file}")
        else:
            print(f"Warning: {zip_file_key}에 해당하는 출력 경로가 없습니다.")

# 2. ZIP 파일을 해제하는 함수
def extract_zip_files(zip_file, extract_to):
    if not os.path.exists(zip_file):
        print(f"ZIP 파일이 존재하지 않습니다: {zip_file}")
        return
    
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"{zip_file} 압축 해제 완료. 압축 해제 경로: {extract_to}")

# 3. Training 라벨링 데이터 병합 및 압축 해제
merge_part_files(train_part_folder, train_output_zip_files)

# Training ZIP 파일 해제
for zip_file, output_path in train_output_zip_files.items():
    extract_zip_files(output_path, os.path.join(train_zip_output_folder, os.path.splitext(zip_file)[0]))

# 4. Validation 라벨링 데이터 병합 및 압축 해제
merge_part_files(val_part_folder, val_output_zip_files)

# Validation ZIP 파일 해제
for zip_file, output_path in val_output_zip_files.items():
    extract_zip_files(output_path, os.path.join(val_zip_output_folder, os.path.splitext(zip_file)[0]))

TL_국정감사.zip.part0 병합 완료. ZIP 파일 저장 경로: ./extracted_files/Training/TL_국정감사.zip
TL_본회의.zip.part0 병합 완료. ZIP 파일 저장 경로: ./extracted_files/Training/TL_본회의.zip
TL_소위원회.zip.part0 병합 완료. ZIP 파일 저장 경로: ./extracted_files/Training/TL_소위원회.zip
TL_예산결산특별위원회.zip.part0 병합 완료. ZIP 파일 저장 경로: ./extracted_files/Training/TL_예산결산특별위원회.zip
TL_특별위원회.zip.part0 병합 완료. ZIP 파일 저장 경로: ./extracted_files/Training/TL_특별위원회.zip
./extracted_files/Training/TL_국정감사.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/TL_국정감사
./extracted_files/Training/TL_본회의.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/TL_본회의
./extracted_files/Training/TL_소위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/TL_소위원회
./extracted_files/Training/TL_예산결산특별위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/TL_예산결산특별위원회
./extracted_files/Training/TL_특별위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/TL_특별위원회
VL_국정감사.zip.part0 병합 완료. ZIP 파일 저장 경로: ./extracted_files/Validation/VL_국정감사.zip
VL_본회의.zip.part0 병합 완료. ZIP 파일 저장 경로: ./

In [9]:

# 1. Mecab 형태소 분석기 초기화
mecab = MeCab.Tagger()

# 2. ZIP 파일 압축 해제 함수
def extract_zip_files(zip_folder, extract_to):
    zip_files = [f for f in os.listdir(zip_folder) if f.endswith('.zip')]
    for zip_file in zip_files:
        zip_file_path = os.path.join(zip_folder, zip_file)
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
            print(f"{zip_file_path} 압축 해제 완료. 압축 해제 경로: {extract_to}")

# 3. Mecab을 이용한 형태소 분석 및 전처리
def preprocess_with_mecab(text):
    """Mecab을 사용하여 텍스트를 전처리하는 함수"""
    words = mecab.parse(text).split()  # 형태소 단위로 나누기
    return " ".join(words)

# 4. 데이터 로드 함수
def load_single_file(file_path):
    """한 개의 JSON 파일을 로드하고 형태소 분석을 적용하는 함수"""
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            # 필요한 데이터를 처리
            question = data.get('question', {}).get('comment')  # 질문 필드
            context = data.get('context')  # 문맥 필드
            answer = data.get('answer', {}).get('comment')  # 답변 필드

            # Mecab으로 전처리
            if question:
                question = preprocess_with_mecab(question)
            if context:
                context = preprocess_with_mecab(context)
            if answer:
                answer = preprocess_with_mecab(answer)

            # answer_start는 문맥 내에서 답변 시작 위치를 찾음
            answer_start = context.find(answer) if answer in context else -1

            if question and context and answer and answer_start != -1:
                return (question, context, answer, answer_start)
        except json.JSONDecodeError as e:
            print(f"JSON 디코딩 오류: {e}, 파일: {file_path}")
    return None

# 5. 캐시 관련 함수
CACHE_FILE = 'data_cache.pkl'

def load_data_from_cache(cache_file):
    """캐시된 데이터를 불러오는 함수"""
    if os.path.exists(cache_file):
        print(f"캐시된 데이터를 불러오는 중: {cache_file}")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    return None

def save_data_to_cache(data, cache_file):
    """데이터를 캐시에 저장하는 함수"""
    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)
    print(f"데이터를 캐시에 저장했습니다: {cache_file}")

# 6. 병렬로 데이터 로드
def load_data_parallel(data_folder):
    """여러 개의 파일을 병렬로 로드하는 함수"""
    qa_pairs = []
    print(f"데이터 폴더 경로: {data_folder}")

    # 파일 리스트 가져오기
    files = [os.path.join(root, file)
             for root, _, files in os.walk(data_folder)
             for file in files if file.endswith('.json')]

    print(f"발견된 파일들: {len(files)}개")

    # 각 파일을 병렬로 로드
    from concurrent.futures import ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(load_single_file, files))

    # None이 아닌 결과만 모으기
    qa_pairs = [result for result in results if result is not None]

    print(f"로드된 QA 쌍 개수: {len(qa_pairs)}")
    return qa_pairs

# 7. 캐시와 함께 데이터 로드 함수
def load_data_with_cache(data_folder, cache_file):
    """캐시를 사용해 데이터를 로드하는 함수"""
    # 캐시에서 데이터를 불러오려고 시도
    data = load_data_from_cache(cache_file)
    if data is not None:
        return data

    # 캐시된 데이터가 없으면 병렬로 데이터 로드
    data = load_data_parallel(data_folder)
    save_data_to_cache(data, cache_file)
    return data

# 8. 데이터 경로 설정 및 압축 해제
train_zip_folder = './extracted_files/Training'
val_zip_folder = './extracted_files/Validation'

# 압축 해제 경로
train_extracted_path = './extracted_files/Training/unzipped'
val_extracted_path = './extracted_files/Validation/unzipped'

# ZIP 파일 압축 해제
extract_zip_files(train_zip_folder, train_extracted_path)
extract_zip_files(val_zip_folder, val_extracted_path)

# 캐시된 데이터를 로드 (없을 경우 병렬로 로드 후 캐시에 저장)
train_qa_pairs = load_data_with_cache(train_extracted_path, 'train_data_cache.pkl')
val_qa_pairs = load_data_with_cache(val_extracted_path, 'val_data_cache.pkl')

# 데이터가 비어있는지 확인
print(f"Training 데이터 쌍 개수: {len(train_qa_pairs)}")
print(f"Validation 데이터 쌍 개수: {len(val_qa_pairs)}")

if len(train_qa_pairs) == 0 or len(val_qa_pairs) == 0:
    raise ValueError("데이터셋이 비어 있습니다. 데이터 로드 또는 전처리를 확인하세요.")

# 9. BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")

# 10. Dataset 및 DataLoader 설정
class QADataset(Dataset):
    def __init__(self, qa_pairs, tokenizer, max_len=512):
        self.qa_pairs = qa_pairs
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        question, context, answer, answer_start = self.qa_pairs[idx]

        # 토크나이즈하고 필요한 인코딩 준비
        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        # 정답에 대한 시작과 끝 위치 계산
        answer_end = answer_start + len(self.tokenizer.encode(answer, add_special_tokens=False))

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        token_type_ids = inputs["token_type_ids"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            "start_positions": torch.tensor(answer_start),
            "end_positions": torch.tensor(answer_end)
        }

train_dataset = QADataset(train_qa_pairs, tokenizer)
val_dataset = QADataset(val_qa_pairs, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# 11. 모델 학습 및 검증
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(data_loader)

# 12. 학습 루프
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = evaluate(model, val_loader, device)
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

# 13. Gradio 웹 인터페이스 구현
def predict_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    start = torch.argmax(start_scores)
    end = torch.argmax(end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start:end]))
    return answer

def chatbot_interface(user_input):
    context = "국회 회의록에서의 발언 내용"  # 실제 문맥으로 대체해야 함
    answer = predict_answer(user_input, context)
    return answer

# 14. Gradio 웹 인터페이스 실행
interface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="BERT 기반 국회 회의록 챗봇")
interface.launch()


./extracted_files/Training/TL_국정감사.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/unzipped
./extracted_files/Training/TL_본회의.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/unzipped
./extracted_files/Training/TL_소위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/unzipped
./extracted_files/Training/TL_예산결산특별위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/unzipped
./extracted_files/Training/TL_특별위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Training/unzipped
./extracted_files/Validation/VL_국정감사.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Validation/unzipped
./extracted_files/Validation/VL_본회의.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Validation/unzipped
./extracted_files/Validation/VL_소위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Validation/unzipped
./extracted_files/Validation/VL_예산결산특별위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Validation/unzipped
./extracted_files/Validation/VL_특별위원회.zip 압축 해제 완료. 압축 해제 경로: ./extracted_files/Validation/unzipped
데이터 폴더 경로: ./extract

KeyboardInterrupt: 

In [ ]:
import os
import json
import pickle
import torch
from transformers import BertTokenizer, BertForQuestionAnswering, AdamW
from torch.utils.data import Dataset, DataLoader
import gradio as gr
from concurrent.futures import ThreadPoolExecutor
import MeCab

# 1. Mecab 형태소 분석기 초기화
mecab = MeCab.Tagger()

# 2. Mecab을 이용한 형태소 분석 및 전처리
def preprocess_with_mecab(text):
    """Mecab을 사용하여 텍스트를 전처리하는 함수"""
    words = mecab.parse(text).split()
    return " ".join(words)

# 3. 데이터 로드 함수
def load_single_file(file_path):
    """한 개의 JSON 파일을 로드하고 형태소 분석을 적용하는 함수"""
    with open(file_path, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            # 필요한 데이터를 처리
            question = data.get('question', {}).get('comment')  # 질문 필드
            context = data.get('context')  # 문맥 필드
            answer = data.get('answer', {}).get('comment')  # 답변 필드

            # Mecab으로 전처리
            if question:
                question = preprocess_with_mecab(question)
            if context:
                context = preprocess_with_mecab(context)
            if answer:
                answer = preprocess_with_mecab(answer)

            # answer_start는 문맥 내에서 답변 시작 위치를 찾음
            answer_start = context.find(answer) if answer in context else -1

            if question and context and answer and answer_start != -1:
                return (question, context, answer, answer_start)
        except json.JSONDecodeError as e:
            print(f"JSON 디코딩 오류: {e}, 파일: {file_path}")
    return None

# 4. 캐시 관련 함수
CACHE_FILE = 'data_cache.pkl'

def load_data_from_cache(cache_file):
    """캐시된 데이터를 불러오는 함수"""
    if os.path.exists(cache_file):
        print(f"캐시된 데이터를 불러오는 중: {cache_file}")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    return None

def save_data_to_cache(data, cache_file):
    """데이터를 캐시에 저장하는 함수"""
    with open(cache_file, 'wb') as f:
        pickle.dump(data, f)
    print(f"데이터를 캐시에 저장했습니다: {cache_file}")

# 5. 병렬로 데이터 로드
def load_data_parallel(data_folder):
    """여러 개의 파일을 병렬로 로드하는 함수"""
    qa_pairs = []
    print(f"데이터 폴더 경로: {data_folder}")

    # 파일 리스트 가져오기
    files = [os.path.join(root, file)
             for root, _, files in os.walk(data_folder)
             for file in files if file.endswith('.json')]

    print(f"발견된 파일들: {len(files)}개")

    # 각 파일을 병렬로 로드
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(load_single_file, files))

    # None이 아닌 결과만 모으기
    qa_pairs = [result for result in results if result is not None]

    print(f"로드된 QA 쌍 개수: {len(qa_pairs)}")
    return qa_pairs

# 6. 캐시와 함께 데이터 로드 함수
def load_data_with_cache(data_folder, cache_file):
    """캐시를 사용해 데이터를 로드하는 함수"""
    # 캐시에서 데이터를 불러오려고 시도
    data = load_data_from_cache(cache_file)
    if data is not None:
        return data

    # 캐시된 데이터가 없으면 병렬로 데이터 로드
    data = load_data_parallel(data_folder)
    save_data_to_cache(data, cache_file)
    return data

# 7. 데이터 경로 설정
train_extracted_path = './extracted_files/Training/unzipped'
val_extracted_path = './extracted_files/Validation/unzipped'

# 캐시된 데이터를 로드 (없을 경우 병렬로 로드 후 캐시에 저장)
train_qa_pairs = load_data_with_cache(train_extracted_path, 'train_data_cache.pkl')
val_qa_pairs = load_data_with_cache(val_extracted_path, 'val_data_cache.pkl')

# 데이터가 비어있는지 확인
print(f"Training 데이터 쌍 개수: {len(train_qa_pairs)}")
print(f"Validation 데이터 쌍 개수: {len(val_qa_pairs)}")

if len(train_qa_pairs) == 0 or len(val_qa_pairs) == 0:
    raise ValueError("데이터셋이 비어 있습니다. 데이터 로드 또는 전처리를 확인하세요.")

# 8. BERT 모델 및 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForQuestionAnswering.from_pretrained("bert-base-multilingual-cased")

# 9. Dataset 및 DataLoader 설정
class QADataset(Dataset):
    def __init__(self, qa_pairs, tokenizer, max_len=512):
        self.qa_pairs = qa_pairs
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.qa_pairs)

    def __getitem__(self, idx):
        question, context, answer, answer_start = self.qa_pairs[idx]

        # 토크나이즈하고 필요한 인코딩 준비
        inputs = self.tokenizer.encode_plus(
            question,
            context,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        # 정답에 대한 시작과 끝 위치 계산
        answer_end = answer_start + len(self.tokenizer.encode(answer, add_special_tokens=False))

        input_ids = inputs["input_ids"].squeeze()
        attention_mask = inputs["attention_mask"].squeeze()
        token_type_ids = inputs["token_type_ids"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "token_type_ids": token_type_ids,
            "start_positions": torch.tensor(answer_start),
            "end_positions": torch.tensor(answer_end)
        }

train_dataset = QADataset(train_qa_pairs, tokenizer)
val_dataset = QADataset(val_qa_pairs, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# 10. 모델 학습 및 검증
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)

def train_epoch(model, data_loader, optimizer, device):
    model.train()
    total_loss = 0
    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        token_type_ids = batch["token_type_ids"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, start_positions=start_positions, end_positions=end_positions)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    return total_loss / len(data_loader)

def evaluate(model, data_loader, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device)
            start_positions = batch["start_positions"].to(device)
            end_positions = batch["end_positions"].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, start_positions=start_positions, end_positions=end_positions)
            loss = outputs.loss
            total_loss += loss.item()

    return total_loss / len(data_loader)

# 11. 학습 루프
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    train_loss = train_epoch(model, train_loader, optimizer, device)
    val_loss = evaluate(model, val_loader, device)
    print(f"Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

# 12. Gradio 웹 인터페이스 구현
def predict_answer(question, context):
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt", max_length=512, truncation=True)
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        start_scores = outputs.start_logits
        end_scores = outputs.end_logits

    start = torch.argmax(start_scores)
    end = torch.argmax(end_scores) + 1

    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][start:end]))
    return answer

def chatbot_interface(user_input):
    context = "국회 회의록에서의 발언 내용"  # 실제 문맥으로 대체해야 함
    answer = predict_answer(user_input, context)
    return answer

# 13. Gradio 웹 인터페이스 실행
interface = gr.Interface(fn=chatbot_interface, inputs="text", outputs="text", title="BERT 기반 국회 회의록 챗봇")
interface.launch()


데이터 폴더 경로: ./extracted_files/Training/unzipped
발견된 파일들: 35233개
로드된 QA 쌍 개수: 8594
데이터를 캐시에 저장했습니다: train_data_cache.pkl
데이터 폴더 경로: ./extracted_files/Validation/unzipped
발견된 파일들: 4400개
로드된 QA 쌍 개수: 1081
데이터를 캐시에 저장했습니다: val_data_cache.pkl
Training 데이터 쌍 개수: 8594
Validation 데이터 쌍 개수: 1081


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/user/miniforge3/envs/nlp/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Token indices sequence length is longer than the specified maximum sequence length for this model (1523 > 512). Running this sequence through the model will result in indexing errors
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation stra